In [2]:
from bs4 import BeautifulSoup
import re
import requests
import json

In [3]:
headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}
headers = {}
data = {}

In [4]:
def crawl_prothom_alo_into_jsob(start_date_str, end_date_str):
    from datetime import datetime, timedelta
    base_url = "https://www.prothomalo.com"
    archive_url = base_url+"/archive/"
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    delta = end_date - start_date
    j = -1
    per_data = {}
    for i in range(delta.days + 1):
        date = start_date + timedelta(days = i)
        url = archive_url+date.strftime("%Y-%m-%d")
        infile = open('newspaper/prothom_alo_'+ date.strftime("%Y-%m-%d") + ".txt", "w+", encoding = 'utf-8')
        while url:
            j = j + 1
            print("reading url >>> " + url)
            try:
                html_content = requests.get(url, headers = headers).content
            except:
                print('Connection Problem for url: ', url, '. Skipping it.')
                break
            soup = BeautifulSoup(html_content, "lxml")
            links = soup.findAll('a', {'class': 'link_overlay'}, href = True)
            for link in links:
                try:
                    inner_html_content = requests.get(base_url+link['href'], headers = headers).content
                except:
                    print('Connection Problem for article, url: ', base_url+link['href'])
                    continue
                inner_soup = BeautifulSoup(inner_html_content, "lxml")
                articleBody = inner_soup.find('div', {'itemprop': 'articleBody'})
                if articleBody:
                    headline = inner_soup.find('div', {'class': 'right_title'}).text
#                     print("headline >>> "+headline)
                    per_data[str(j)] = headline + " -> " + str(articleBody.text)
#                     print(per_data["headline"])
#                     per_data["article"] = str(articleBody.text)
#                     infile.write(headline+"\n")
#                     infile.write(str(articleBody.text) + '\n\n')
#                     data.append(per_data)
			
            pagination_div = soup.find('div', {'class': 'pagination'})
            url = None
            if pagination_div:
                next_page_link = pagination_div.find('a', {'class': 'next_page'}, href = True)
                if next_page_link:
                    url = next_page_link['href']
        
        data["content"] = per_data
        infile.write(str(data))
   #     json.dump(data, infile)
        infile.close()

In [5]:
crawl_prothom_alo_into_jsob("2019-01-01","2019-01-02")

reading url >>> https://www.prothomalo.com/archive/2019-01-01
Connection Problem for article, url:  https://www.prothomalo.com/durporobash/article/1572835/%E0%A6%AC%E0%A7%8D%E0%A6%AF%E0%A6%A4%E0%A6%BF%E0%A6%95%E0%A7%8D%E0%A6%B0%E0%A6%AE%E0%A7%80-%E0%A6%86%E0%A7%9F%E0%A7%8B%E0%A6%9C%E0%A6%A8%E0%A7%87-%E0%A6%AC%E0%A7%8D%E0%A6%B0%E0%A6%BF%E0%A6%B8%E0%A6%AC%E0%A7%87%E0%A6%A8%E0%A7%87-%E0%A6%AC%E0%A7%8D%E0%A6%AF%E0%A6%BE%E0%A6%AC%E0%A7%87%E0%A6%B0-%E0%A6%AC%E0%A6%B0%E0%A7%8D%E0%A6%B7%E0%A6%AC%E0%A6%B0%E0%A6%A3
reading url >>> https://www.prothomalo.com/archive/2019-01-01?page=2
reading url >>> https://www.prothomalo.com/archive/2019-01-01?page=3
reading url >>> https://www.prothomalo.com/archive/2019-01-01?page=4
reading url >>> https://www.prothomalo.com/archive/2019-01-01?page=5
reading url >>> https://www.prothomalo.com/archive/2019-01-01?page=6
reading url >>> https://www.prothomalo.com/archive/2019-01-01?page=7
reading url >>> https://www.prothomalo.com/archive/2019-01-02
reading url >